# Project: Data Analysis of Ford GoBike Dataset
### By: Abebe Tarekegne
### Published: on May 10, 2019


## Overview

>Ford GoBike is the Bay Area's new bike share system, with thousands of public bikes for use across San Francisco, East Bay and San Jose. It involves a fleet of specially designed bikes that are locked into a network of docking stations. Bikes can be unlocked from one station and returned to any other station in the system. People use bike share to commute to work or school, run errands, get to appointments, and more. 

>We are going to investigate publicly available datasets to help predict growth of bike ride, or if the use of bike ride differs by city, age, gender, day of the week, hour of the day, weekend.

>The begining section shows how the dataset are gathered and wrangled but one can go directly to:
- Exploratory section from [here](#exploratory)
- Explanatory section from [here](#explanatory)
- Conclusion section from [here](#conclusion)

## Preliminary Wrangling

>We use Ford GoBike's trip Data available for public use [here](https://www.fordgobike.com/system-data)

>`The attributes includes:`
>- trip duration,
>- start/end time and date, 
>- start/end station id, name, lat/long, 
>- bike id, 
>- user type
>- member year of birth, and gender

In [1]:
#import needed packages
import pandas as pd    # For database management
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import zipfile
import os
import io
import requests
import csv
import timeit
from timeit import default_timer as timer

timer_dict = {}
%matplotlib inline

### Download Datasets (.csv and .zip)

>Looking to the list of downloadable files from [here](https://www.fordgobike.com/system-data), it seems that the data available are one csv file from 2017 and the rest are from 2018 and 2019 and all are csv but zip files. I show how promgramatically download those files. 

>First new folder was created and ready to use, then I code to capture both format of the file (.csv and zipped csv) from the link provided.  I used the list created by looking the file names which contains unique numbers from part of the file names itself.

In [98]:
# Make directory in the local working directory if it doesn't already exist
folder_name = 'fordgobike_files'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

#list of unique numbers from filenames
f_lst = list(range(201801,201813)) + [2017, 201901, 201902, 201903]

for f_num in f_lst:
    f_num = str(f_num) #convert integers to string
    if f_num == "2017": # non-zip .csv files
        f_url = 'https://s3.amazonaws.com/fordgobike-data/'+ f_num + '-fordgobike-tripdata.csv'
        temp_file_name = f_num + '-fordgobike-tripdata.csv'
        download = requests.get(f_url,stream=True)    # returns type 'str'
        with open(folder_name +'\\'+ temp_file_name, 'w') as temp_file: # str, hence mode 'w'
            temp_file.write(download.text)
    else:
        f_url = 'https://s3.amazonaws.com/fordgobike-data/'+ f_num + '-fordgobike-tripdata.csv.zip'
        r = requests.get(f_url,stream=True) # check for <Response [200]> good
        check = zipfile.is_zipfile(io.BytesIO(r.content)) 
        if check: # check if zip file and ignore for other formats
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(folder_name)

### Create Master DataFrame

>Now all .csv and .zip files are downloaded to the specified folder name and zip files are unzipped so that all files are in the same format ".csv". 

>We imported iglob from [glob](https://docs.python.org/3/library/glob.html) python module in order to use recursive list comprehension to each files in a given path. [Examples](https://stackoverflow.com/questions/2186525/how-to-use-glob-to-find-files-recursively)

In [2]:
from glob import iglob
import sys

path = r'C:\Users\at9490\Documents\IPython Notebooks\DataAnalyst\project4\fordgobike_files\*.csv' #path
# after reading each csv file and concatenate all assigned to pandas dataframe.
start = timer()
df_master = pd.concat((pd.read_csv(f) for f in iglob(path, recursive=True)), ignore_index=True, sort=False)
end = timer()
timeittake = end - start
timer_dict.update({"csv concat":timeittake})

### Assessing Dataset

In [3]:
# high-level overview of data shape and composition
print("Number of Rows: ",df_master.shape[0])
print("Number of Columns: ",df_master.shape[1])

Number of Rows:  3015214
Number of Columns:  16


####  <center>Original Master Dataset Features</center>

| `Feature Name`          | `Data Type` | `Unique Count` | `NaN Count` |
|-------------------------|-----------|--------------|-----------|
| duration_sec            | int64     | 22152        | 0         |
| start_time              | object    | 3015020      | 0         |
| end_time                | object    | 3015038      | 0         |
| start_station_id        | float64   | 360          | 12437     |
| start_station_name      | object    | 381          | 12437     |
| start_station_latitude  | float64   | 407          | 0         |
| start_station_longitude | float64   | 408          | 0         |
| end_station_id          | float64   | 360          | 12437     |
| end_station_name        | object    | 381          | 12437     |
| end_station_latitude    | float64   | 408          | 0         |
| end_station_longitude   | float64   | 410          | 0         |
| bike_id                 | int64     | 6802         | 0         |
| user_type               | object    | 2            | 0         |
| member_birth_year       | float64   | 92           | 206967    |
| member_gender           | object    | 3            | 206534    |
| bike_share_for_all_trip | object    | 2            | 519700    |

In [4]:
print('Number of duplicates: ',df_master.duplicated().sum())
print()
print('User Type: Number of rides')
print(df_master.user_type.value_counts())
print()
print('Bike Share For All Trip: Number of rides')
print(df_master.bike_share_for_all_trip.value_counts())
print()
print('Member Gender: Number of rides')
print(df_master.member_gender.value_counts())

Number of duplicates:  0

User Type: Number of rides
Subscriber    2544383
Customer       470831
Name: user_type, dtype: int64

Bike Share For All Trip: Number of rides
No     2277409
Yes     218105
Name: bike_share_for_all_trip, dtype: int64

Member Gender: Number of rides
Male      2082988
Female     680192
Other       45500
Name: member_gender, dtype: int64


### Formatting

>`data type convert:` to approprate datatype 
>- 'start_time' str to datetime
>- 'end_time' str to datetime 
>- 'user_type' str to category 
>- 'member_gender' str to category 
>- 'bike_share_for_all_trip' str to category 
>- 'start_station_id' float to int
>- 'end_station_id' float to int
>- 'member_birth_year' float to datetime

>`...more`
>- catagorical variables: 'user_type' and 'member_gender'
>- datetime variables: 'start_time' and 'end_time'
>- integer variables: 'start_station_id' and 'end_station_id'

>Station id, station names, station lat/long we expect to see the same number of unique counts but there were few records with more than one lat/long locations for the same station ID. The same is true for names, as in some cases temporary names were used. For our visualization we will use `station_id` as the bases for referencing.

In [5]:
# Make a copy of original DataFrame
df_clean = df_master.copy()

In [6]:
#Convert from object to category
df_clean['user_type'] = df_clean['user_type'].astype('category')
df_clean['member_gender'] = df_clean['member_gender'].astype('category')
df_clean['bike_share_for_all_trip'] = df_clean['bike_share_for_all_trip'].astype('category')

In [7]:
#Convert objects to datetime
df_clean['start_time']  = pd.to_datetime(df_clean['start_time'])
df_clean['end_time']  = pd.to_datetime(df_clean['end_time'])

#Using datetime I created additional time features
df_clean['start_hour'] = df_clean['start_time'].dt.hour #0 to 22
df_clean['start_date'] = df_clean['start_time'].dt.date 
df_clean['start_weekday'] = df_clean['start_time'].dt.weekday #0 to 6
df_clean['start_day'] = df_clean['start_time'].dt.day
df_clean['start_month'] = df_clean['start_time'].dt.month #0 to ..
df_clean['start_year'] = df_clean['start_time'].dt.year
df_clean['start_weekofyear'] = df_clean['start_time'].dt.weekofyear  # 0 to 52
df_clean['start_weekday_name'] = df_clean['start_time'].dt.strftime('%a') # %a Sun %A Sunday
df_clean['start_month_name'] = df_clean['start_time'].dt.strftime('%b') # %b Jan %B January
df_clean['start_year_month'] = df_clean['start_time'].dt.strftime('%y-%m') # %y 17 or  %Y 2017 

In [8]:
#After replacing NaN values with 0 I convert those listed fields to integers
df_clean[['start_station_id', 'end_station_id','member_birth_year']] = \
df_clean[list(['start_station_id', 'end_station_id','member_birth_year'])].fillna(0).astype(int)

In [9]:
df_clean.shape

(3015214, 26)

In [10]:
df_clean[['start_station_id', 'end_station_id','member_birth_year']].isna().sum()

start_station_id     0
end_station_id       0
member_birth_year    0
dtype: int64

### What is the structure of the dataset?

>There are 3015214 attributes (rides recorded) with 16 original features, and created 9 new features of dates to help for data visualization.

### What are the main feature(s) of interest in the dataset?
>I am most interested in the `duration_sec` but as the start and end oordinates are provided it will be more interesting to find the distance or the average speed of each trip so `new features will be added`.

>And Interesting to investigate the trip duration relationship with:
>- User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual)
>- Member Year of Birth
>- Member Gender

### What type of new features are introduced?

>- Trip Distance
>- Group Station Locations by Cluster of Areas or Cities
>- Member Age Group

#### Trip Distance Calcuation

>Since trip start and end locations are provided I am interested to find the minimum possible distances. With time and distance then we can drive speed.

>We are now creating new feature called `minimum_distance_miles` and `average_speed_mph`

>Let's import geopy module to use for spatial data manuplation. Geopy can calculate geodesic distance between two points using the geodesic distance or the great-circle distance, with a default of the geodesic distance available as the function geopy.distance.distance.m

In [11]:
import geopy
from geopy.distance import geodesic

def distance(origin, destination):
    dist = geopy.distance.geodesic(origin, destination).miles
    return dist

start = timer() #this may take up to 20minutes
# minimum distance traveled from the start to end of the trip
df_clean['minimum_distance_miles'] = df_clean.apply(lambda x: distance((x['start_station_latitude'], x['start_station_longitude']), \
                                                               (x['end_station_latitude'], x['end_station_longitude'])), axis=1)
end = timer()
timeittake = end - start
timer_dict.update({"distance":timeittake})

# trip speed in miles per hour
df_clean['duration_hr'] = df_clean['duration_sec']/3600
df_clean['average_speed_mph'] = df_clean['minimum_distance_miles']/df_clean['duration_hr']

#### Group Station Locations by Cluster of Areas or Cities

>We now create new feature called `city` based on lat/long filtering using:

>- `San Francisco` longitude range(-122.48,-122.35) and latitude range(37.73,37.83)
>- `Oakland` longitude range(-122.32,-122.2) and latitude range(37.77,37.88)
>- `San Jose` longitude range(-122,-121.8) and latitude range(37.3,37.5)

In [12]:
san_index = df_clean.query('(start_station_longitude > -122.48) & (start_station_longitude < -122.35) & \
(start_station_latitude > 37.73) & (start_station_latitude < 37.83)').index
oak_index = df_clean.query('(start_station_longitude > -122.32) & (start_station_longitude < -122.2) & \
     (start_station_latitude > 37.77) & (start_station_latitude < 37.88)').index
saj_index = df_clean.query('(start_station_longitude > -122) & (start_station_longitude < -121.8) & \
     (start_station_latitude > 37.3) & (start_station_latitude < 37.5)').index

In [13]:
df_clean['city']="" # create empty column
df_clean.loc[san_index,'city'] = "San Francisco"
df_clean.loc[oak_index,'city'] = "Oakland"
df_clean.loc[saj_index,'city'] = "San Jose"
# We append all three cities indexes and substract from dataset indexes to get for other part of town
noncity_index = list(set(df_clean.index)-set(san_index.append(oak_index).append(saj_index)))
df_clean.loc[noncity_index,'city'] = "Other"
df_clean['city'] = df_clean['city'].astype('category')
print('City:    Number of rides')
df_clean['city'].value_counts()

City:    Number of rides


San Francisco    2240531
Oakland           625782
San Jose          143945
Other               4956
Name: city, dtype: int64

#### Member Age Group

>We are now creating new feature called `member_age_group` which shows grouping based on the member age range or bins we created. From member_birth_year we use 2019 as reference to find the member_age, and then we use bining to divide in groups of managable size for visualization.

In [14]:
df_clean['member_age'] = 2019 - df_clean.member_birth_year
#replace zero value of member_birth_year which is now shown as 2019 by NaN
df_clean.loc[df_clean['member_age'] == 2019, 'member_age'] = np.nan
#Create bins based on the range of member_age
bins = [10, 20, 30, 40, 50, 60, 70, 100, 140]
df_clean['member_age_group'] = pd.cut(df_clean.member_age,bins, \
                                      labels=["10-20","20-30","30-40","40-50","50-60","60-70","70-100","100-140"] )
print('Age Group: Number of rides')
df_clean['member_age_group'].value_counts().sort_index()

Age Group: Number of rides


10-20        33381
20-30       926499
30-40      1096387
40-50       443560
50-60       235526
60-70        60494
70-100       10579
100-140       1818
Name: member_age_group, dtype: int64

In [15]:
print('Stats for Duration in minutes')
print((df_clean['duration_sec']/60).describe(percentiles = [.1, .5,.75, .99]))

Stats for Duration in minutes
count    3.015214e+06
mean     1.466674e+01
std      4.170826e+01
min      1.016667e+00
10%      3.833333e+00
50%      9.283333e+00
75%      1.455000e+01
99%      1.011500e+02
max      1.439483e+03
Name: duration_sec, dtype: float64


In [16]:
print('Stats for Distance in miles')
df_clean['minimum_distance_miles'].describe(percentiles = [.1, .5,.75, .99])

Stats for Distance in miles


count    3.015214e+06
mean     1.004740e+00
std      6.493740e-01
min      0.000000e+00
10%      3.447281e-01
50%      8.665367e-01
75%      1.310504e+00
99%      3.033850e+00
max      4.316416e+01
Name: minimum_distance_miles, dtype: float64

#### What the Statistics tells us from two variables `duration_sec`, and `minimum_distance_miles`?

>- 75% of the rides took less than 15 minutes and travel less than 1.4 miles
>- 99% of the rides took less than 102 minutes and travel less than 3.1 miles
>- Average trip ride duration and distance are 12.35 minutes and 1 mile respectively

#### Removing Outliers

In [17]:
df_clean.size

96486848

In [18]:
#Now let's remove outlies when we plot the distribution of each
df_clean = df_clean[(df_clean.minimum_distance_miles <= 4)&(df_clean.duration_hr <= 3)]

In [19]:
df_clean.shape

(2994564, 32)

Export the dataframe to .csv for later use

In [20]:
# As we already created new features, including distance and city we are exporting the data into csv
df_clean.to_csv('master_data.csv')

In [21]:
df = pd.read_csv('master_data.csv',low_memory=False)

In [22]:
df.shape

(2994564, 33)

In [23]:
# Here we show some sample of the dataset within the file size limit of Github (25Mb)
df.sample(75000).to_csv('master_75k.csv')

Go to `2_FordGoBike_Exploratory_Part1` [here](https://github.com/abebe64/FordGoBike/2_FordGoBike_Exploratory_Part1.ipynb)